In [1]:
import requests as rq
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from TCGA_files import get_tcga_tissue
#from ensembleAPI import geneinfo, genesinfo
from sklearn import metrics
from hsbmpy import *
import os

In [ ]:
import importlib, hsbmpy
importlib.reload(hsbmpy)
from hsbmpy import *

## query many

In [2]:
level = 1
algorithm = 'topsbm'
directory = "/home/fvalle/phd/datasets/cancers/luad/"
directory = "/home/jovyan/work/phd/datasets/cancers/crectal"
L=get_max_available_L(directory, algorithm)
df_clusters = pd.read_csv("%s/%s/%s_level_%d_clusters.csv"%(directory,algorithm,algorithm,L), header=[0])
df_clusters.head()

,Cluster 1
0,07ec290d-c41b-42eb-8cb2-07504451f732.htseq.cou...
1,3388b247-719d-41d6-a600-83abd6bd6204.htseq.cou...
2,11586a7f-5148-4b6b-b431-686b907f8df9.htseq.cou...
3,d65a7cb6-4b1f-4976-990e-c4f47ae11986.htseq.cou...
4,fd40f0e5-0825-409b-8a4c-86da09f0a84c.htseq.cou...


In [3]:
#df_files = pd.read_csv("https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt", sep='\t')
#df_files.set_index('SAMPID', inplace=True)
#df_files.dropna(how='all', axis=0).to_csv("%s/files.dat"%directory, index=True, header=True)

In [4]:
df_files = pd.read_csv("%s/files.dat"%directory, index_col=[0], header=[0]).dropna(axis=1, how='all').dropna(axis=0, how='all')
#df_files.insert(2,'disease_tissue',["%s[%s]"%(df_files.at[s,'primary_site'],df_files.at[s,'disease_type']) for s in df_files.index])
labels=df_files.columns
#df_files.groupby(['primary_site','disease_type']).count()
labels

Index(['tcga_id', 'cases.0.diagnoses.0.morphology',
       'cases.0.diagnoses.0.primary_diagnosis',
       'cases.0.diagnoses.0.tissue_or_organ_of_origin',
       'cases.0.diagnoses.0.tumor_grade', 'cases.0.diagnoses.0.tumor_stage',
       'cases.0.project.disease_type', 'cases.0.project.primary_site',
       'cases.0.samples.0.longest_dimension',
       'cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id',
       'cases.0.submitter_id', 'index', 'sequenced', 'microRNA', 'SNP6', 'WGS',
       'RNAseq', 'Agilent.expression', 'methylation', 'normal_meth',
       'complete', 'MSI_status', 'methylation_subtype', 'MLH1_silencing',
       'expression_subtype', 'hypermutated', 'silent_mutrate',
       'nonsilent_mutrate', 'cancer', 'anatomic_organ_subdivision',
       'tumor_site', 'age_at_initial_pathologic_diagnosis', 'days_to_death',
       'days_to_last_followup', 'days_to_last_known_alive',
       'distant_metastasis_pathologic_spread', 'gender', 'histological_type',
       

In [8]:
df_files

,tcga_id,cases.0.diagnoses.0.morphology,cases.0.diagnoses.0.primary_diagnosis,cases.0.diagnoses.0.tissue_or_organ_of_origin,cases.0.diagnoses.0.tumor_grade,cases.0.diagnoses.0.tumor_stage,cases.0.project.disease_type,cases.0.project.primary_site,cases.0.samples.0.longest_dimension,cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id,...,number_of_lymphnodes_positive_by_he,person_neoplasm_cancer_status,preoperative_pretreatment_cea_level,primary_tumor_pathologic_spread,prior_diagnosis,residual_tumor,synchronous_colon_cancer_present,tumor_stage,vascular_invasion_present,vital_status
file_name,,,,,,,,,,,,,,,,,,,,,
21981535-9a4b-44ca-a3c0-3f52f4c0d29b.FPKM.txt.gz,TCGA-A6-2671,8140/3,"Adenocarcinoma, NOS",Sigmoid colon,not reported,stage iv,Colon Adenocarcinoma,Colorectal,1.8,TCGA-A6-2671-11A-01R-A32Z-07,...,3,WITH TUMOR,17.4,T3,NO,R0,NO,Stage IV,YES,LIVING
d15c1117-5ccd-47da-bb0a-51e5a71535ff.FPKM.txt.gz,TCGA-A6-2671,8140/3,"Adenocarcinoma, NOS",Sigmoid colon,not reported,stage iv,Colon Adenocarcinoma,Colorectal,1.8,TCGA-A6-2671-01A-01R-1410-07,...,3,WITH TUMOR,17.4,T3,NO,R0,NO,Stage IV,YES,LIVING
59e2ab4e-9529-490d-b12c-9b7f24c8166c.FPKM.txt.gz,TCGA-A6-2672,8140/3,"Adenocarcinoma, NOS","Colon, NOS",not reported,stage iiib,Colon Adenocarcinoma,Colorectal,unknown,TCGA-A6-2672-01B-03R-2302-07,...,1,TUMOR FREE,3.4,T3,NO,R0,NO,Stage IIIB,YES,LIVING
1b054b39-aa38-4356-8447-71f211365a01.FPKM.txt.gz,TCGA-A6-2672,8140/3,"Adenocarcinoma, NOS","Colon, NOS",not reported,stage iiib,Colon Adenocarcinoma,Colorectal,1.3,TCGA-A6-2672-01A-01R-0826-07,...,1,TUMOR FREE,3.4,T3,NO,R0,NO,Stage IIIB,YES,LIVING
ef8d5ef0-1834-4a2d-a916-4862ce9dc136.FPKM.txt.gz,TCGA-A6-2674,8480/3,Mucinous adenocarcinoma,Sigmoid colon,not reported,stage iv,Colon Adenocarcinoma,Colorectal,0.8,TCGA-A6-2674-01A-02R-0821-07,...,7,TUMOR FREE,32.8,T3,NO,R0,NO,Stage IV,YES,LIVING
7f9a629b-12ed-48cc-8d5c-1c2f5db9cf1f.FPKM.txt.gz,TCGA-A6-2674,8480/3,Mucinous adenocarcinoma,Sigmoid colon,not reported,stage iv,Colon Adenocarcinoma,Colorectal,unknown,TCGA-A6-2674-01B-04R-A277-07,...,7,TUMOR FREE,32.8,T3,NO,R0,NO,Stage IV,YES,LIVING
424d8e5f-9fc6-470b-ad2c-b4447b0eb07e.FPKM.txt.gz,TCGA-A6-2674,8480/3,Mucinous adenocarcinoma,Sigmoid colon,not reported,stage iv,Colon Adenocarcinoma,Colorectal,0.8,TCGA-A6-2674-01A-02R-A278-07,...,7,TUMOR FREE,32.8,T3,NO,R0,NO,Stage IV,YES,LIVING
4ea4ed16-8597-426c-9485-0fc8a8d81973.FPKM.txt.gz,TCGA-A6-2676,8140/3,"Adenocarcinoma, NOS","Colon, NOS",not reported,stage iib,Colon Adenocarcinoma,Colorectal,0.9,TCGA-A6-2676-01A-01R-0826-07,...,0,TUMOR FREE,6.3,unknown,NO,R0,NO,Stage IIB,NO,LIVING
97503aa9-93a5-4ff6-bb19-203ea235c2bf.FPKM.txt.gz,TCGA-A6-2677,8140/3,"Adenocarcinoma, NOS","Colon, NOS",not reported,stage iiic,Colon Adenocarcinoma,Colorectal,0.7,TCGA-A6-2677-01A-01R-0821-07,...,5,TUMOR FREE,27.4,T3,NO,R0,NO,Stage IIIC,NO,LIVING


In [ ]:
labels=['primary_site', 'disease_type']
#labels = ['subtype', 'pathologic_stage', 'vital_status','days_to_death_binned']
labels = ['primary_site', 'disease_type', 'primary_diagnosis', 
       'methylation_subtype', 'expression_subtype',
       'hypermutated', 'cancer',
       'anatomic_organ_subdivision', 'tumor_site',
       'age_at_initial_pathologic_diagnosis', 'days_to_death',
       'distant_metastasis_pathologic_spread', 'gender', 'histological_type',
       'history_of_colon_polyps',     'lymphatic_invasion_present', 'lymphnode_pathologic_spread',
       'person_neoplasm_cancer_status', 'preoperative_pretreatment_cea_level',
       'primary_tumor_pathologic_spread', 'prior_diagnosis', 'residual_tumor',
       'synchronous_colon_cancer_present', 'tumor_stage',
       'vascular_invasion_present', 'vital_status'] #crectal

#labels = ['cases.0.diagnoses.0.morphology',
#       'cases.0.diagnoses.0.primary_diagnosis',
#       'cases.0.diagnoses.0.tissue_or_organ_of_origin',
#       'cases.0.diagnoses.0.tumor_grade', 'cases.0.diagnoses.0.tumor_stage',
#       'cases.0.project.disease_type', 'cases.0.project.primary_site',
#       'cases.0.samples.0.longest_dimension'] #lung
#labels = ['subtype']
#labels=['SMTS', "SMTSD"]

In [ ]:
for normalise in [True, False]:
    for label in labels:
        for level in np.arange(L+1)[::-1]:
            if level==0:
                #pass
                continue
            print(normalise, label, level)
            try:
                cluster = get_cluster_given_l(level, directory,algorithm=algorithm)
                fraction_sites = get_fraction_sites(cluster,df_files=df_files,label=label, normalise=normalise)

                #fsdf = pd.DataFrame(data=fraction_sites)
                #fsdf = fsdf.drop('Other', axis=1)
                #fsdf = fsdf.divide(fsdf.sum(axis=1), axis=0).fillna(0)
                #fraction_sites = fsdf.sort_values(by=fsdf.columns.to_list(), ascending=True).to_dict(orient='list')

                clustersinfo = get_clustersinfo(cluster,fraction_sites)
                plot_cluster_composition(fraction_sites,directory,level,label=label, normalise=normalise,algorithm=algorithm)
                make_heatmap(fraction_sites, directory, label, level, normalise=normalise,algorithm=algorithm)

                clustersinfo = get_clustersinfo(cluster,fraction_sites)            
                if not normalise:
                    plot_maximum(clustersinfo,cluster,label,level, directory,algorithm=algorithm)
                    plot_maximum_size(clustersinfo,label,level, directory,algorithm=algorithm)
                    plot_maximum_label(clustersinfo,label,level, directory,algorithm=algorithm)
                    plot_sizes(level,directory, algorithm=algorithm)
            except:
                print(*sys.exc_info())
            try:
                fraction_sites_shuffle = get_fraction_sites(cluster, pd.read_csv("%s/files_shuffles.dat"%directory, index_col=[0]),label=label, normalise=normalise)
                clustersinfo_shuffle = get_clustersinfo(cluster, fraction_sites_shuffle)
                plot_cluster_composition(fraction_sites_shuffle,directory,level, label=label, shuffled=True, normalise=normalise)
                if not normalise:
                    plot_maximum(clustersinfo,cluster,label,level,directory,clustersinfo_shuffle,algorithm=algorithm)
                    plot_maximum_size(clustersinfo,label,level, directory,clustersinfo_shuffle,algorithm=algorithm)
                    plot_maximum_label(clustersinfo,label,level, directory,clustersinfo_shuffle,algorithm=algorithm)
                    plot_labels_size(clustersinfo,label,level, directory,clustersinfo_shuffle,algorithm=algorithm)
            except:
                print("must shuffle files")

In [ ]:
##useful for R clustree plot
def reindex(x):
    i=1
    last=x[0]
    new = []
    for xi in x:
        if xi == last:
            pass
        else:
            i+=1
            last=xi
        new.append(i)
    return new

df_clusters = pd.read_csv("%s/%s/%s_level_%d_clusters.csv"%(directory,algorithm,algorithm,L), header=[0])
df_labels = pd.DataFrame()
shape = df_clusters.dropna().shape
files = df_clusters.dropna().values.reshape(shape[0]*shape[1],)
for level in np.arange(L+1)[::-1]:
    print(level)
    df_clusters = pd.read_csv("%s/%s/%s_level_%d_clusters.csv"%(directory,algorithm,algorithm,level), header=[0])
    currentlevellabels = []
    for file in files:
        s = df_clusters[df_clusters.isin([file])].any(0)
        a = s.index[s]
        currentlevellabels.append(int(a[0][8:])-1)
    df_labels.insert(0,'l%d'%level,currentlevellabels)
    del currentlevellabels
filelabels = []
filesublabels = []
for file in files:
    try:
        filelabels.append(get_file(file, df_files)[labels[0]])
        filesublabels.append(get_file(file, df_files)[labels[1]])
    except:
        filelabels.append('unknown')
        filesublabels.append('unknown')
        print(*sys.exc_info())
df_labels.insert(0,'tissue', filelabels)
df_labels.insert(0,'subtissue', filesublabels)
df_labels.sort_values(by=['tissue','subtissue'], inplace=True)
df_labels.sort_values(by=["l%d"%l for l in np.arange(L+1)[:0:-1]], axis=0, inplace=True)
filelabels = df_labels['tissue']
filesublabels = df_labels['subtissue']
df_labels = df_labels.apply(reindex, axis=0)
df_labels['tissue']=filelabels
df_labels['subtissue']=filesublabels
df_labels.to_csv("%s/%s/topsbm_labels.csv"%(directory,algorithm), index=False, header=True)

In [ ]:
if "clustersizes.txt" in os.listdir(directory):
    with open("%s/clustersizes.txt"%directory, 'r') as f:
        xl=np.array(f.read().split()).astype(int)
else:
    try:
        xl = getclustersizesarray(directory, L)
        with open("%s/clustersizes.txt" % directory, 'w') as f:
            for x in xl:
                f.write("%d\n" % x)
    except:
        print(*sys.exc_info())
        print("cannot save clustersizes.txt")

if "topicsizes.txt" in os.listdir(directory):
    with open("%s/topicsizes.txt"%directory) as f:
        tl=np.array(f.read().split()).astype(int)
else:
    try:
        tl = gettopicsizesarray(directory, L)
        with open("%s/topicsizes.txt" % directory, 'w') as f:
            for x in tl:
                f.write("%d\n" % x)
    except:
        print("cannot save topicsizes.txt")

In [ ]:
scores = get_scores(directory, labels, verbose=False)
scores['hsbm'] = scores['SMTS']

In [ ]:
fig=plt.figure(figsize=(18,15))
ax = fig.subplots(1)
#add_score_lines(ax,scores,[labels[0]],xl, h=True, c=True, alpha=1)
h = np.array(scores['hsbm']['h'])
c = np.array(scores['hsbm']['c'])
ax.plot(scores['hsbm']['xl'],h, marker='o', ls='--', lw=10, ms=40, label='homogeneity')
ax.plot(scores['hsbm']['xl'],c, marker='o', ls='-.', lw=10,ms=40, label='completeness')
ax.plot(scores['hsbm']['xl'],scores['hsbm']['V'], marker='o', ms=40, ls='-', lw=10, label='score')
ax.set_xscale('log')
ax.set_ylim(0,1)
#ax.plot(xl, 2*h*c/(h+c), ls='-',c='g')
plt.xticks(fontsize=35)
plt.yticks(fontsize=35)
plt.legend(fontsize=35)

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=15)

plt.xlabel('number of clusters', fontsize=35)
plt.ylabel('measure', fontsize=35)
plt.show()
fig.savefig("%s/metric_scores_primarysite.pdf"%(directory))

In [ ]:
label = "SMTS"
scores = {}
scores['hsbm'] = get_scores(directory, labels, df_files=df_files, verbose=False)[label]
scores['hierarchical'] = get_scores(directory, labels, algorithm='hierarchical', verbose=False)[label]
scores['lda'] = get_scores(directory, labels, algorithm='lda', verbose=False)[label]
#scores['kmeans'] = get_scores(directory, labels, algorithm='kmeans', verbose=False)[label]
#scores['tm']=get_scores(directory, labels, algorithm='tm', verbose=False)[label]
#scores['cc']=get_scores(directory, labels, algorithm='cc', verbose=False)[label]
#scores['wgcna']=get_scores(directory, labels, algorithm='wgcna', verbose=False)['SMTS']
scores['shuffle'] = get_scores_shuffled(directory, df_files, label=label, algorithm='topsbm')

In [ ]:
scores['hierarchical-tpm'] = get_scores(directory, labels, algorithm='hierarchical-tpm', verbose=False)[label]
scores['lda-tpm'] = get_scores(directory, labels, algorithm='lda-tpm', verbose=False)[label]

In [ ]:
fig=plt.figure(figsize=(20,15))
ax = fig.subplots(1)
add_score_lines(ax,scores,labels=['hsbm', 'tm', 'hierarchical', 'lda', 'hierarchical-tpm', 'lda-tpm', 'wgcna', 'shuffle'])
#ax.set_xscale('linear')
plt.xlim(1,2000)
#plt.legend(ncol=2, fontsize=35,loc='upper left' )
plt.show()
fig.savefig("%s/metric_scores_all.pdf"%(directory))

In [ ]:
scores["lda"]

In [ ]:
scores["lda"]